<a href="https://colab.research.google.com/github/atmark-techno/armadillo-iot-g4_model_zoo/blob/main/GoogleColabNotebooks/SuperResolution_Quantize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## 必要なライブラリのインポート

In [1]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_hub as hub

# 量子化

## 量子化時に必要なデータセットを取得

ダウンロードするデータセットのサイズが大きいので、実行に時間がかかります。

In [2]:
raw_test_data, info = tfds.load(name="div2k", with_info=True, split='train')
data = raw_test_data.take(100)

EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/800 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/div2k/bicubic_x2/2.0.0.incomplete82O4W7/div2k-train.tfrecord*...:   0%|       …

Generating validation examples...:   0%|          | 0/100 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/div2k/bicubic_x2/2.0.0.incomplete82O4W7/div2k-validation.tfrecord*...:   0%|  …

Dataset div2k downloaded and prepared to ~/tensorflow_datasets/div2k/bicubic_x2/2.0.0. Subsequent calls will reuse this data.


## 量子化時に必要になる代表データセットを返す関数を定義

In [3]:
def representative_dataset():
  for example in data:
    image = example['hr'].numpy()
    image = Image.fromarray(image)
    width, height = image.size
    image = image.resize(
        (input_size, input_size),
        Image.BICUBIC)
    image = np.asarray(image)
    image = tf.cast(image, dtype=tf.float32)
    image = image[tf.newaxis, :]
    yield [image]

## tfhubからモデルをロード

デフォルトで入力サイズが不定(-1)となっていますが、このままtfliteに変換するとエラーとなるため、入力サイズを予め固定します。

In [4]:
input_size = 50

model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, input_size, input_size, 3])

## 量子化の実行と保存

In [5]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model = converter.convert()

with open('super_resolution.tflite', 'wb') as f:
  f.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
